1. Combine the three different files to form a single file, having entries from each file.

In [ ]:
import pandas as pd
import numpy as np
impcat=pd.read_csv('C:/Users/hp/Desktop/impcat_data_june2020.csv')
pdp=pd.read_csv('C:/Users/hp/Desktop/pdp_data_june2020.csv')
ga=pd.read_csv('C:/Users/hp/Desktop/ga_data_new_sept20.csv')

In [ ]:
df = impcat.append(pdp, sort=False)
result = df.append(ga, sort=False)
print(result.columns)
fin=pd.DataFrame()

In [ ]:
result.head()

2. Keep only the mentioned columns: ['Query','Impressions','Avg_Position','GLCAT_MCAT_ID','GLCAT_MCAT_NAME','Total Unique Searches']

In [ ]:
result=result[['Query','Impressions','Avg_Position','GLCAT_MCAT_ID','GLCAT_MCAT_NAME','Total Unique Searches']]
result.head()

3. For each query in a MCAT: 
   a.  If only one entry for a query exists (in a MCAT) calculate and store the following as:
       data[impression]=data[‘Impressions’]
       data[unique_searches]= data[‘Unique Searches’]
       data[avg_position]= data['Avg_Position']
   b.If more than one entry for a query exists (in a MCAT) calculate and store the following as:
     data[impression]= Sum of Impressions
     data[unique_searches]= Sum of Unique Searches
     data[avg_position]= Average of 'Avg_Position'

In [ ]:
#tup=tuple(result.groupby(['GLCAT_MCAT_ID','Query']))
#dictionary = {} 
pairs=dict(tuple(result.groupby(['GLCAT_MCAT_ID','Query'])))
#group= dict((x, y) for x, y in tup)
#group= dictionary(Convert(tup, dictionary))
print (len(pairs))

In [ ]:
r=0
for x,y in pairs.items():
    print (x)
    y=y.reset_index(drop=True)
    mcat_name=str(y['GLCAT_MCAT_NAME'][0])
    y['Impressions']=y['Impressions'].fillna(0)
    y['Avg_Position']=y['Avg_Position'].fillna(0)
    y['Total Unique Searches']=y['Total Unique Searches'].fillna(0)
    mcat_id=int(x[0])
    query=str(x[1])
    if (len(y)==1):
        sum_impression=y['Impressions'][0]
        sum_average_position=y['Avg_Position'][0]
        sum_unique_searches=y['Total Unique Searches'][0]
    else:
        sum_impression=sum(y['Impressions'])
        impression_list=list(y['Impressions'])
        average_position_list=list(y['Avg_Position'])
        #try:
        if (sum_impression!=0):
            sum_average_position = np.average(average_position_list, weights=impression_list)
        else:
            sum_average_position=0
        sum_unique_searches=sum(y['Total Unique Searches'])
    fin.at[r,"MCAT_ID"]=mcat_id
    fin.at[r,"MCAT_NAME"]=mcat_name
    fin.at[r,"Query"]=query
    fin.at[r,"IMPRESSIONS"]=sum_impression
    fin.at[r,"AVG POSITION"]=sum_average_position
    fin.at[r,"UNIQUE_SEARCHES"]=sum_unique_searches
    r+=1
    

1. Columns after this step: mcat_id, mcat_name, query,impression, avg_position, unique_searches
2. Save the results in a file (combined_data.csv)

In [ ]:
fin.to_csv('C:/Users/hp/Desktop/combined_data.csv',index=False)

In [ ]:
fin.head()

For each query, we need to clean it by applying the following steps:
○	Conversion to lowercase
○	Keeping only alphabets and numeric entries
○	Removal of all english stop words
○	Removal of certain keywords mentioned below:
["manufacturer","manufacture","manufacturing","manufactured","price","wholesaler","wholesalers","wholesale","deal","dealer","deals","dealers","distribution","supplier","distributor","distributors","india","rate","cost","costs","near me","buy","buys","online","company",  "exporter","exporters","good","topmost","business","trusted","finest","offer","organization","trader","organizations","traders","offers","indiamart","delhi","manufacturers","prices","suppliers","manufactures"]

○	Removal of location names provided in the pickle file (location.pkl)
○	Singularize the query
○	Replace one than one spaces with a single space
○	Trim the extra spaces (trailing and preceding)


In [ ]:
import nltk
import re 
import pickle 
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
from pattern.en import singularize
from nltk.corpus import stopwords
all_stopwords = stopwords.words('english')

sw_1=["manufacturer","manufacture","manufacturing","manufactured","price","wholesaler","wholesalers","wholesale","deal","dealer","deals","dealers","distribution","supplier","distributor","distributors","india","rate","cost","costs","near me","buy","buys","online","company",
      "exporter","exporters","good","topmost","business","trusted","finest","offer","organization","trader","organizations","traders","offers","indiamart","delhi","manufacturers","prices","suppliers","manufactures"]

location = list(pickle.load(open(r'C:\Users\hp\Desktop\location_set.pkl','rb')))

In [ ]:
def clean(text):
    text=str(text)
    text=text.lower()
    x=re.sub('[^A-Za-z0-9]+', ' ', text)
    x=x.strip()
    tokens_sw1 = ' '.join(word for word in  w_tokenizer.tokenize(x) if not word in all_stopwords)
    tokens_sw2 = ' '.join(word for word in  w_tokenizer.tokenize(tokens_sw1) if not word in sw_1)
    tokens_sw3 = ' '.join(word for word in  w_tokenizer.tokenize(tokens_sw2) if not word in location)
    stem_text=' '.join(singularize(plural) for plural in w_tokenizer.tokenize(tokens_sw3))
    tokens_sw4 = ' '.join(word for word in  w_tokenizer.tokenize(stem_text) if not word in sw_1)
    x1 = re.sub(' +', ' ',tokens_sw4)
    x1=x1.strip()
    return x1

This will give us a clean_query corresponding to each query in a MCAT.
Columns after this step: mcat_id, mcat_name, query, impression, avg_position, unique_searches, query_cleaned
Save the results in a file (cleaned_queries.csv)

In [ ]:
result["Query_Cleaned"]=result["Query"].apply(clean)
result.head()

In [ ]:
result.to_csv(r'C:\Users\hp\Desktop\cleaned_queries.csv',index=False)

Now, for each clean_query in a MCAT (please note that many queries when cleaned might result in the same clean_query), we will calculate and store the following:
○	data[‘Impressions’]= Sum of Impressions
○	data[‘unique_search’]= Sum of Unique Searches
○	data[‘frequency’]= Count of the times ‘clean_query’ appears in a MCAT
○	If the clean_query exists as it is in the original data, we take the value of “avg_position” for our clean_query, else we take the Minimum of average_position
○	For a MCAT, sort the values as per ‘Impressions’ of queries.(high to low)

In [ ]:
groups=dict(tuple(result.groupby(['MCAT_ID'])))
print (len(groups))

In [ ]:
result1_1=pd.DataFrame()
for key,value in groups.items():
    result_1=pd.DataFrame()
    row=0
    print (key)
    value=value.reset_index()
    mcat_name=str(value['MCAT_NAME'][0])
    queries=list(set(value['Query_Cleaned']))
    for i in range(0,len(queries)):
        print (queries[i])
        df1=value[value['Query_Cleaned']==queries[i]]
        df1=df1.reset_index()
        df1['IMPRESSIONS'] = df1['IMPRESSIONS'].fillna(0)
        df1['UNIQUE_SEARCHES'] = df1['UNIQUE_SEARCHES'].fillna(0)
        sum_impression=sum(df1['IMPRESSIONS'])
        sum_searches=sum(df1['UNIQUE_SEARCHES'])
        
        rank=df1[df1['Query']==queries[i]]
        rank=rank.reset_index(drop=True)
        if (len(rank)>=1):
            pos=rank['AVG POSITION'][0]
        else:
            pos_list=list(df1['AVG POSITION'])
            try:
                pos=min(i for i in pos_list if i!=0)
            except:
                pos=0
        
        del (rank)
        freq=len(df1)
        result_1.at[row,"MCAT_ID"]=int(key)
        result_1.at[row,"MCAT_NAME"]=mcat_name
        result_1.at[row,"Query"]=str(queries[i])
        result_1.at[row,"FREQUENCY"]=int(freq)
        result_1.at[row,"SUM_IMPRESSIONS"]=sum_impression
        result_1.at[row,"POSITION"]=pos
        result_1.at[row,"SUM_UNIQUE_SEARCHES"]=sum_searches
        row=row+1
    result_1=result_1.sort_values(by=['SUM_IMPRESSIONS'],ascending=False)
    result_1=result_1[result["Query"]!=""]
    result_1=result_1.reset_index()
    result1_1=result1_1.append(result_1)
    print(result1_1.shape)

Columns after this step: mcat_id, mcat_name, query (cleaned_query), frequency, sum_impression, position, sum_unique_searches
Save the results in a file (cleaned_queries_with_freq.csv)

In [ ]:
result1_1.to_csv(r'C:\Users\hp\Desktop\cleaned_queries_with_freq.csv',index=False)

For each MCAT:
○	Find all the uni-grams.
○	For each uni-gram, calculate the total sum_impressions (summation of “sum_impression” values for every entry where the uni-gram exists)
○	Return top-15 for every MCAT. (as per “sum_impression”- high to low)
○	Save the results in a file (MCAT_wise_uni_grams.csv)

In [ ]:
# get unigrams
result1_1=pd.DataFrame()
for key,value1 in groups.items():
    Tokens = []
    Unique_List=list(value1['Query_Cleaned'].astype(str))
    for i in range(len(Unique_List)):
        temp_w = Unique_List[i].split(' ')
        Tokens = Tokens + temp_w
        tokens_without_sw = [word for word in Tokens if not word in all_stopwords]
    DG_df = pd.DataFrame(data=tokens_without_sw,columns=['KW'])
    DG_df=DG_df[DG_df["KW"]!='']
    uni_grams=list(DG_df["KW"])
    for i in range(0,len(uni_grams)):
        value1["Query1"]=" "+value1["Query_Cleaned"].astype(str)+" "
        l=" "+str(uni_grams[i])+" "
        a=value1[value1["Query1"].str.contains(l)]
        a=a.reset_index()
        a["IMPRESSIONS"]=a["IMPRESSIONS"].fillna(0)
        sum_impression=sum(a["IMPRESSIONS"])
        DG_df.loc[DG_df.KW == uni_grams[i], 'Sum_Impressions'] = sum_impression
    DG_df["MCAT_ID"]=int(key)
    DG_df=DG_df.sort_values(by=['Sum_Impressions'],ascending=False)
    DG_df=DG_df.drop_duplicates()
    DG_df=DG_df.reset_index()
    DG_df=DG_df.iloc[:15]
    result1_1=result1_1.append(DG_df)
    print (result1_1.shape)
result1_1.to_csv(r'C:\Users\hp\Desktop\MCAT_wise_uni_grams.csv',index=False)

○Find all the bi-grams.
○For each bi-gram, calculate the total sum_impressions (summation of “sum_impression” values for every entry where the bi-gram exists)
○Return top-15 for every MCAT. (as per “sum_impression”- high to low)
○Save the results in a file (MCAT_wise_bi_grams.csv)

In [ ]:
# get bigrams
import pandas as pd
result1_1=pd.DataFrame()
for key,value1 in groups.items():
    Tokens = []
    Unique_List=list(value1['Query_Cleaned'].astype(str))
    for i in range(len(Unique_List)):
        temp_w = Unique_List[i].split(' ')
        temp_w_1=[word for word in temp_w if not word in all_stopwords]
        if(len(temp_w_1) >=2):
            for j in range(len(temp_w_1)-1):
                w1 = str(temp_w_1[j]) + " " + str(temp_w_1[j+1])
                Tokens = Tokens + [w1]
    DG_df = pd.DataFrame(data=Tokens,columns=['KW'])
    bi_grams=list(DG_df["KW"])
    for i in range(0,len(bi_grams)):
        value1["Query1"]=" "+value1["Query_Cleaned"].astype(str)+" "
        l=" "+str(bi_grams[i])+" "
        a=value1[value1["Query1"].str.contains(l)]
        a=a.reset_index()
        a["IMPRESSIONS"]=a["IMPRESSIONS"].fillna(0)
        sum_impression=sum(a["IMPRESSIONS"])
        DG_df.loc[DG_df.KW == bi_grams[i], 'Sum_Impressions'] = sum_impression
    DG_df["MCAT_ID"]=int(key)
    DG_df=DG_df.sort_values(by=['Sum_Impressions'],ascending=False)
    DG_df=DG_df.drop_duplicates()
    DG_df=DG_df.reset_index()
    # top 15
    DG_df=DG_df.iloc[:15]
    result1_1=result1_1.append(DG_df)
    print (result1_1.shape)
result1_1.to_csv(r'C:\Users\hp\Desktop\MCAT_wise_bi_grams.csv',index=False)